In [194]:
import pandas as pd													# data manipulation
import matplotlib.pyplot as plt												# graphs
import seaborn as sns													# graphs
import numpy as np													# graphs
from scipy.stats import norm												# statistics
from sklearn.preprocessing import StandardScaler									# statistics
from scipy import stats													# statistics
from math import *													# advance calculation	
from matplotlib import cm												# graphs
import sys														# system functions
import warnings														# indicating warnings
warnings.filterwarnings('ignore')											# ignore by default
from matplotlib.ticker import LinearLocator										# graphs
pd.set_option('display.max_columns', None)										# show all columns
pd.set_option('display.max_rows', None)											# show all rows
np.set_printoptions(threshold=sys.maxsize)										# show arrays full
pd.set_option("display.max_colwidth", -1)										# show a complete text in a cell
from xlsxwriter.workbook import Workbook										# CSV writing
import csv														# CSV writing	
import xgboost as xgb													# prediction
import pickle														# prediction	
import sklearn														# mutual information
import statsmodels.api as sm												# coefficient
from sklearn.ensemble import RandomForestClassifier									# random forest
from sklearn import linear_model											# linear regression
import pymysql														# MySql connection
import datetime														# time conversions
import random														# randomchoice

In [50]:
df = pd.read_csv('df_mercadona.csv')

In [51]:
df.drop(columns=['supermarket'],inplace=True)
df['insert_date'] = pd.to_datetime(df['insert_date'])

In [43]:
for column in df.columns:
    print(column, ':', df.duplicated(subset=[column]).sum())

id : 0
category : 1002715
name : 997127
price : 1002000
reference_price : 1001674
reference_unit : 1003012
insert_date : 1002862


In [193]:
# Average price and ref price by cat

dfresult =pd.DataFrame(data={'Category':[''],'Product':[''],'PriceTagAVG':[''],'PriceRefAVG':[''],'PriceRefDiff%':[''],'Period':[''],'Change':['']},
                        columns=['Category','Product','PriceTagAVG','PriceRefAVG','PriceRefDiff%','Period','Change'])

index=0
for name in df['name'].sample(2000).unique():
    dfresult.at[index, 'Category'] = df.loc[df['name'] == name]['category'].value_counts(ascending=False).index[0]
    dfresult.at[index, 'Product'] = name
    dfresult.at[index, 'PriceTagAVG'] = round(df.loc[df['name'] == name]['price'].mean(),2)
    dfresult.at[index, 'PriceRefAVG'] = round(df.loc[df['name'] == name]['reference_price'].mean(),2)
    PriceRefDiff = round(((df.loc[df['name'] == name]['reference_price'].max())-(df.loc[df['name'] == name]['reference_price'].min()))/(df.loc[df['name'] == name]['reference_price'].min())*100,2)
    
    ifpart1 = round(((df.loc[(df['name'] == name) & (df['reference_price'] == df.loc[df['name'] == name]['reference_price'].min())]['insert_date'].max()-df.loc[(df['name'] == name) & (df['reference_price'] == df.loc[df['name'] == name]['reference_price'].max())]['insert_date'].min())/ np.timedelta64(1, 'D')),0)
    ifpart2 = round(((df.loc[(df['name'] == name) & (df['reference_price'] == df.loc[df['name'] == name]['reference_price'].min())]['insert_date'].min()-df.loc[(df['name'] == name) & (df['reference_price'] == df.loc[df['name'] == name]['reference_price'].max())]['insert_date'].max())/ np.timedelta64(1, 'D')),0)
    if abs(ifpart1) > abs(ifpart2):
        dfresult.at[index, 'Period'] = abs(ifpart1)
        dfresult.at[index, 'Change'] = change = 'Decrease'
    elif (abs(ifpart1) == 0) & (abs(ifpart2) == 0):
        dfresult.at[index, 'Period'] = 0
        dfresult.at[index, 'Change'] = 'NA'
    elif abs(ifpart1) == abs(ifpart2):
        dfresult.at[index, 'Period'] = abs(ifpart1)
        dfresult.at[index, 'Change'] = 'NA2'
    else:
        dfresult.at[index, 'Period'] = abs(ifpart2)
        dfresult.at[index, 'Change'] = change = 'Increase'
    
    if PriceRefDiff == 0:
        dfresult.at[index, 'Change'] = change = 'Steady'
        
    if change =='Decrease':
        dfresult.at[index, 'PriceRefDiff%'] = -abs(PriceRefDiff)
    elif change == 'Increase':
        dfresult.at[index, 'PriceRefDiff%'] = PriceRefDiff
    elif change == 'Steady':
        dfresult.at[index, 'PriceRefDiff%'] = 0

    index = index+1
dfresult = dfresult.sort_values(['Category', 'Product'], ascending=[True, True])
dfresult.drop(dfresult.loc[(dfresult['Change'] == 'NA') | (dfresult['Change'] == 'NA2')].index,inplace=True)
dfresult

,Category,Product,PriceTagAVG,PriceRefAVG,PriceRefDiff%,Period,Change
0,perfume_y_colonia,Body spray citronela Deliplus,1.5,1,0,78,Steady
2,huevos,Huevos de codorniz,1.19,0.79,0,78,Steady
3,coloracion_cabello,Coloración permanente Garnier 110 rubio extra claro ceniza,3.65,3.65,0,78,Steady
4,cuidado_corporal,Crema corporal anticelulítica Deliplus,2.3,0.92,0,78,Steady
5,utensilios_de_limpieza_y_calzado,Plantillas gel talla 35-40 Bosque Verde,4,2,0,78,Steady
...,...,...,...,...,...,...,...
1651,cuidado_corporal,Loción corporal hydra & go Deliplus,3.6,1.8,0,78,Steady
1652,biberon_chupete_y_menaje,Tetina silicona tres posiciones +1 mes Deliplus,3,1.5,-1.35,78,Decrease
1653,limpieza_muebles_y_multiusos,Limpiador plata y metales Bosque Verde en crema,2.3,9.2,0,78,Steady
1654,pasta_y_fideos,Pasta piñones Hacendado,0.69,1.38,0,78,Steady
